In [25]:
import os, subprocess
import json
import uproot
import awkward as ak
import numpy as np
from coffea import processor, util, hist

%matplotlib inline
import matplotlib.pyplot as plt
from cycler import cycler

import mplhep as hep
plt.style.use([hep.style.ROOT, hep.style.CMS])

In [26]:
colors = {}
colors['QCD'] = '#1f77b4'
colors['VBF'] = '#ff7f0e'
colors['VV'] = '#2ca02c'
colors['Wjets'] = '#d62728'
colors['WH'] = '#9467bd'
colors['Zjets'] = '#8c564b'
colors['ZH'] = '#e377c2'
colors['ggF'] = '#7f7f7f'
colors['ttbar'] = '#bcdb22'
colors['singlet'] = '#bcdb22'
colors['ttH'] = '#17becf'

In [27]:
lumis = {}
lumis['2016'] = 35.9
lumis['2017'] = 41.1
lumis['2018'] = 59.9

nfiles_mc = {}
nfiles_mc['2016'] = 113
nfiles_mc['2017'] = 167
nfiles_mc['2018'] = 201

nfiles_data = {}
nfiles_data['2016'] = 101
nfiles_data['2017'] = 97
nfiles_data['2018'] = 133

with open('xsec.json') as f:
    xs = json.load(f)

with open('pmap.json') as f:
    pmap = json.load(f)

In [28]:
year = '2016'
outsum = processor.dict_accumulator()

In [29]:
# Load all MC
for n in range(1,nfiles_mc[year]+1):
    print(n)
    filename = 'condor/outfiles/'+year+'_'+str(n)+'.coffea'
    if os.path.isfile(filename):
        out = util.load(filename)
        outsum.add(out)
    else:
        print("File " + filename + " is missing")
        
scale_lumi = {k: xs[k] * 1000 *lumis[year] / w for k, w in outsum['sumw'].items()}
outsum['cutflow'].scale(scale_lumi, 'dataset')
outsum['templates1'].scale(scale_lumi, 'dataset')
outsum['templates2'].scale(scale_lumi, 'dataset')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113


In [30]:
# Load all data
for n in range(1,nfiles_data[year]+1):
    print(n)
    filename = 'condor/outdata/'+year+'_'+str(n)+'.coffea'
    if os.path.isfile(filename):
        out = util.load(filename)
        outsum.add(out)
    else:
        print("File " + filename + " is missing")
        
# BE CAREFUL NOT TO UNBLIND
# Can you figure out how to blind it at this step???

1
2
3
4


ValueError: Cannot add this histogram with histogram <Hist (dataset,region,genflavor,cut) instance at 0x7f5d9275cc10> of dissimilar dimensions

In [ ]:
templates = outsum['cutflow'].group('dataset', hist.Cat('process', 'Process'), pmap)

In [ ]:
cutflow = templates.sum('genflavor').integrate('region','muoncontrol').integrate('process','muondata')
cutflow.values()

In [ ]:
cutflow = templates.sum('genflavor').integrate('region','muoncontrol').integrate('process','ttbar')
cutflow.values()

In [ ]:
templates1 = outsum['templates1'].group('dataset', hist.Cat('process', 'Process'), pmap).integrate('region','muoncontrol')

In [ ]:
hist.plot1d(templates1.sum('pt1').integrate('process','muondata'),overlay='ddb1')

In [ ]:
templates2 = outsum['templates2'].group('dataset', hist.Cat('process', 'Process'), pmap).integrate('region','muoncontrol')

In [ ]:
hist.plot1d(templates2.sum('msd1','etamu').integrate('process','muondata'),overlay='ddb1')

In [ ]:
hist.plot1d(templates2.sum('msd1','ptmu').integrate('process','muondata'),overlay='ddb1')

In [ ]:
hist.plot1d(templates2.sum('ptmu','etamu').integrate('process','muondata'),overlay='ddb1')

In [ ]:
hist.plot1d(templates2.sum('ptmu','etamu').integrate('process','ttbar'),overlay='ddb1')

In [ ]:
def plot_stack(x,name):
    x.label = 'Events'
    axes = hist.plot1d(x, overlay='process', fill_opts={'edgecolor': (0,0,0,1)}, stack=True, order=['ttH','ZH','WH','VBF','ggF','VV','ttbar','singlet','Wjets','Zjets','QCD'])
    axes.set_prop_cycle(cycler(color=colors.values()))
    axes.set_yscale('log')
    axes.set_ylim(.001, 100000)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    #png_name = year+'/plot-all/'+name+'_stack.png'
    #plt.savefig(png_name,bbox_inches='tight')
    #pdf_name = year+'/plot-all/'+name+'_stack.pdf'
    #plt.savefig(pdf_name,bbox_inches='tight')

In [ ]:
plot_stack(templates2.sum('ptmu','etamu').integrate('ddb1',int_range=slice(0,0.89)),'msd1')

In [ ]:
plot_stack(templates2.sum('ptmu','etamu').integrate('ddb1',int_range=slice(0.89,1)),'msd1')

In [ ]:
plot_stack(templates2.sum('msd1','etamu').integrate('ddb1',int_range=slice(0,0.89)),'ptmu')

In [ ]:
plot_stack(templates2.sum('msd1','etamu').integrate('ddb1',int_range=slice(0.89,1)),'ptmu')

In [ ]:
plot_stack(templates2.sum('msd1','ptmu').integrate('ddb1',int_range=slice(0,0.89)),'etamu')

In [ ]:
plot_stack(templates2.sum('msd1','ptmu').integrate('ddb1',int_range=slice(0.89,1)),'etamu')